In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain-community langchain-teddynote

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
! pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from peft import PeftModel

from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever

In [ ]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)

        # Retriever 생성
        # retriever = db.as_retriever(search_type="mmr",
        #                             search_kwargs={'k': 3, 'fetch_k': 8})


        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()

        retriever = EnsembleRetriever(
            retrievers=[kiwi_bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5],
            search_type="mmr",
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/dataset/dacon_langchain

/content/drive/MyDrive/dataset/dacon_langchain


In [ ]:
debug = True

In [ ]:
base_directory = './' # Your Base Directory

if debug:
  df = pd.read_csv('./train.csv')
  len = df.shape[0]//2
  print(len)
  df = df.iloc[:len].reset_index(drop = True)
else:
  df = pd.read_csv('./test.csv')


df.head()

248


,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."


In [ ]:
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권...


Processing PDFs:  33%|███▎      | 1/3 [00:09<00:18,  9.19s/it]

Processing 2024 나라살림 예산개요...


Processing PDFs:  67%|██████▋   | 2/3 [00:23<00:12, 12.38s/it]

Processing 재정통계해설...


Processing PDFs: 100%|██████████| 3/3 [00:33<00:00, 11.18s/it]


In [ ]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    model_id = "/content/drive/MyDrive/model/ko-gemma-2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

#     model = PeftModel.from_pretrained(model, "./persona/checkpoint-200",is_trainable=True)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        #temperature=0.2,
        return_full_text=False,
        max_new_tokens=512,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

In [ ]:
# LLM 파이프라인
llm = setup_llm_pipeline()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
template = """
다음 정보를 바탕으로 질문에 답하세요:
{context}

질문: {question}

주어진 질문에만 답변하세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
답변:
"""

In [ ]:
# template = """
# 다음 정보를 바탕으로 질문에 답하세요:
# {context}

# 질문: {question}

# 주어진 질문에만 답변하세요. 문장으로 답변해주세요. 단답도 해주세요.
# 답변:
# """

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
total_rows = df.shape[0]
for _, row in tqdm(df.iterrows(), total=total_rows, desc="Answering Questions"):

    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성

    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/248 [00:00<?, ?it/s]

Question: 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?


Answering Questions:   0%|          | 1/248 [00:07<28:58,  7.04s/it]

Answer: 2024년 중앙정부 재정체계는 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.

Question: 2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?


Answering Questions:   1%|          | 2/248 [00:15<31:46,  7.75s/it]

...

Answer: 이상기후로 인한 수해 관련 재해 대응을 위해 빗물을 일시에 저장하여 도심 저지대 침수피해를 예방하기 위한 우수저류시설 6개소가 신규 설치될 예정이며, 하상도로에 하천 수위계 센서에 연계하여 자동으로 출입을 통제하는 시설도 전국 180개소에 구축될 예정입니다. 




Question: 대학의 일반재정지원을 확대하는 이유와 국립대학 및 사립대학의 투자 확대 방식은 무엇인가?


Answering Questions:  23%|██▎       | 57/248 [09:07<57:27, 18.05s/it]

Answer: 대학의 일반재정지원을 확대하는 이유는 등록금 동결, 물가 상승 등에 대응하여 대학이 충분한 교육을 제공할 수 있도록 돕기 위해서입니다. 국립대학은 생활안전, 첨단 기자재 중심으로 투자를 확대하고, 사립대학은 경영자문 컨설팅을 지원하여 체계적 구조개선을 촉진합니다. 




Question: 2024년 R&D 분야에 대한 재정투자 규모는 2023년 대비 얼마나 감소했으며, 어떤 분야의 국가전략기술 투자를 확대할 계획인가?


Answering Questions:  23%|██▎       | 58/248 [09:21<53:21, 16.85s/it]

Answer: 2024년 R&D 분야에 대한 재정투자 규모는 2023년 대비 9.5% 감소한 26.5조원입니다. 2024년에는 AI, 첨단바이오, 반도체, 이차전지 등 혁신기술의 내재화에 집중 투자할 계획입니다. 




Question: 우주산업 클러스터 삼각체제 구축사업의 목표와 국가 우주산업 육성 거점 지역은 어디인가?


In [ ]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())

    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

In [ ]:
if debug:

  submit_df = pd.DataFrame()
  submit_df['Answer'] = [item['Answer'] for item in results]

  # submit_df = pd.read_csv('baseline_submission.csv')
  pred = submit_df['Answer']

  # df = pd.read_csv('train.csv')
  gt = df['Answer']

  result = calculate_average_f1_score(gt, pred)
  print(result)

else:
  # 제출용 샘플 파일 로드
  submit_df = pd.read_csv("./sample_submission.csv")

  # 생성된 답변을 제출 DataFrame에 추가
  submit_df['Answer'] = [item['Answer'] for item in results]
  submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

  # 결과를 CSV 파일로 저장
  submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)